immanquablement des petites corrections a faire quand on traite les auteurs

# 1er episode : 26 janv 2025

et voir le contenu de `scripts/store_all_auteurs_from_episode.py`

In [ ]:
from mongo_episode import Episode
from mongo_auteur import Auteur, AuthorChecker

In [ ]:
import datetime

date = datetime.datetime(2025, 1, 26)

In [ ]:
import sys
import pandas as pd

episode = Episode.from_date(date=date)
if not episode:
    print(f"Episode du {date.strftime('%d/%m/%Y')} n'existe pas")
    # stoppe la cellule mais continue le notebook
    # raise RuntimeError("Arrêt de l'exécution de la cellule")
    # stoppe le notebook
    sys.exit(1)

affiche_episode = f"""
Date: {episode.date.strftime("%d %b %Y")}
Titre: {episode.titre}
Description: {episode.description}

"""
print(affiche_episode)

auteurs = episode.get_all_auteurs()
print(f"La liste des auteurs : {auteurs}\n")

ac = AuthorChecker(episode)

auteur_traitement_df = pd.DataFrame(
    columns=["auteur_corrige", "detection", "existait_en_base", "anomalie"]
)
verbose = False
analyse_dict = {}
for auteur in auteurs:
    auteur_corrige_dict = ac.check_author(auteur, return_details=True, verbose=verbose)
    if verbose:
        print(auteur_corrige_dict)
    # check_author retourne None dans author_corrected si le process via rss:metadata, db, llm, web search n'a rien renvoye
    # c'est donc une anomalie
    anomalie = True if auteur_corrige_dict["author_corrected"] is None else False
    # dans auteur_corrige c'est le nom de l'auteur si il est different de l'original sinon None
    auteur_corrige = (
        auteur_corrige_dict["author_corrected"]
        if auteur_corrige_dict["author_corrected"]
        != auteur_corrige_dict["author_original"]
        else None
    )
    nom_auteur = auteur_corrige_dict["author_corrected"] if not None else auteur
    # a quel endroit a ete detecte l'auteur (rss:metadata, db, llm, web search)
    detection = auteur_corrige_dict["source"]
    existait_en_base = False if anomalie else Auteur(nom_auteur).exists()
    auteur_traitement_df.loc[auteur] = [
        auteur_corrige,
        detection,
        existait_en_base,
        anomalie,
    ]
    if not anomalie:
        aut = Auteur(nom_auteur)
        aut.keep()
    else:
        analyse_dict[auteur] = {
            "analyse": auteur_corrige_dict["analyse"],
            "score": auteur_corrige_dict["score"],
        }


display(auteur_traitement_df)
if len(analyse_dict) > 0:
    print("\nAnalyse des anomalies\n")
for auteur, details in analyse_dict.items():
    print(f"{auteur} -> {details['analyse']} (score: {details['score']})")


Date: 26 Jan 2025
Titre: Les nouveaux romans de Leïla Slimani, Pierre Lemaître, Jeanne Rivière, Nathalie Azoulai, Milena Agus
Description: durée : 00:47:56 - Le Masque et la Plume - par : Rebecca Manzoni - Une saga familiale à travers trois générations de femmes, entre le Maroc et la France ; une histoire d'amour et une réflexion sur la judéité ; un roman filial et d'espionnage dans la Guerre Froide ; amitié, désir, musique punk sans les années 90 ; littérature et amour en Sardaigne. - invités : Arnaud Viviant, Laurent CHALUMEAU, Patricia Martin, Elisabeth Philippe - Arnaud Viviant : Critique littéraire (Revue Regards), Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier, Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault


La liste des auteurs : ['Pierre Lemaitre', 'Leila Slimani', 'Nathalie Azoulay', 'Jeanne Rivière', 'Milena Agus', 'Haruki Muraka

,auteur_corrige,detection,existait_en_base,anomalie
Pierre Lemaitre,Pierre Lemaître,rss:metadata,True,False
Leila Slimani,Leïla Slimani,rss:metadata,True,False
Nathalie Azoulay,Nathalie Azoulai,rss:metadata,True,False
Jeanne Rivière,None,rss:metadata,True,False
Milena Agus,None,rss:metadata,True,False
Haruki Murakami,None,mongodb:auteurs,True,False
Constantin Alexandrakis,None,mongodb:auteurs,True,False
Vanessa Springora,None,mongodb:auteurs,True,False
Neige Sinault,None,web search,False,True
Jean Echenoz,None,mongodb:auteurs,True,False



Analyse des anomalies

Neige Sinault -> La requête Google ne contient aucune information indiquant que Neige Sinault est un auteur de livres. Les résultats mentionnent principalement Philippe Sinault, qui est lié au sport automobile, et d'autres occurrences du nom Sinault dans des contextes variés, mais aucun ne fait référence à Neige Sinault en tant qu'auteur. Par conséquent, il n'y a pas de preuve que Neige Sinault soit un auteur de livres. (score: 0)


on va corriger Neige Sinno

In [ ]:
neige = Auteur("Neige Sinno")
neige.keep()

In [ ]:
vir_desp = Auteur("Virginie Despentes")
vir_desp.exists()

False

# index ou pas ?

In [ ]:
from mongo_auteur import Auteur
import timeit


def load_from_db():
    return Auteur.get_entries()


# On utilise timeit.repeat pour exécuter la fonction 10 fois
# number=1 exécute le code une seule fois par répétition
durations = timeit.repeat(
    "load_from_db()", setup="from __main__ import load_from_db", repeat=10, number=1
)

# Calcul de la durée moyenne
moyenne = sum(durations) / len(durations)
print(f"Temps moyen d'exécution load_from_db: {moyenne:.6f} secondes")

Temps moyen d'exécution load_from_db: 3.677106 secondes


In [ ]:
from mongo import get_collection

auteurs_collection = get_collection(collection_name="auteurs")

# Créer un index sur le champ "nom"
index_name = auteurs_collection.create_index([("nom", 1)])
print(f"Index créé: {index_name}")

Index créé: nom_1


In [ ]:
from mongo_auteur import Auteur
import timeit


def load_from_db():
    return Auteur.get_entries()


# On utilise timeit.repeat pour exécuter la fonction 10 fois
# number=1 exécute le code une seule fois par répétition
durations = timeit.repeat(
    "load_from_db()", setup="from __main__ import load_from_db", repeat=10, number=1
)

# Calcul de la durée moyenne
moyenne = sum(durations) / len(durations)
print(f"Temps moyen d'exécution load_from_db apres index : {moyenne:.6f} secondes")

Temps moyen d'exécution load_from_db apres index : 3.422320 secondes


bon ca change rien mais je ne fais pas de requete avec le nom donc c'est sans doute logique

# test d'homonymie

Pour la recherche d'homonymes (c'est-à-dire comparer des noms d'après leur prononciation plutôt que leur orthographe exacte), TheFuzz (basé sur la distance de Levenshtein) n'est pas forcément adapté. À la place, vous pouvez utiliser des algorithmes de traitement phonétique qui transforment les chaînes en codes phonétiques afin de comparer leur sonorité.

Quelques solutions possibles :

• Double Metaphone / Metaphone
Ces algorithmes sont souvent utilisés pour comparer la prononciation en anglais, mais ils peuvent être adaptés au français. Il existe par exemple la librairie fuzzy ou jellyfish qui implémente Double Metaphone. Vous pourriez comparer les codes phonétiques de "Neige Sinault" et "Neige Sinno" pour détecter leur similarité.

• Soundex adapté au français (ou Phonex)
Certaines implémentations de Soundex ont été adaptées pour le français (parfois appelées « Soundex français » ou « Phonex »). Ces solutions ne sont pas toujours disponibles "out-of-the-box" en python, mais vous pouvez trouver des implémentations ou en écrire une en vous basant sur l'algorithme.

Pour résumer, il n'existe pas de librairie équivalente à TheFuzz dédiée spécifiquement à la recherche d'homonymes en français, mais vous pouvez combiner un prétraitement (comme la normalisation des accents avec unidecode) avec un algorithme phonétique (Double Metaphone ou Soundex adapté) pour obtenir ce type de matching.

Exemple avec jellyfish et Double Metaphone :

```python
import jellyfish

def are_phonetically_similar(name1: str, name2: str) -> bool:
    # Récupérer le premier code double metaphone (le second code est optionnel)
    code1 = jellyfish.double_metaphone(name1)[0]
    code2 = jellyfish.double_metaphone(name2)[0]
    return code1 == code2

# Exemple
name1 = "Neige Sinault"
name2 = "Neige Sinno"

if are_phonetically_similar(name1, name2):
    print("Les noms se prononcent de manière similaire")
else:
    print("Les noms sont différents phoniquement")
```

Jellyfish ne fournit pas directement un "score d’homonymie" à partir de l’algorithme Double Metaphone, puisque ce dernier retourne des codes phonétiques plutôt que des valeurs numériques. Cependant, Jellyfish offre d’autres fonctions de similarité (comme jaro_winkler_similarity, damerau_levenshtein_distance ou levenshtein_distance) qui permettent d’obtenir un score numérique indiquant la similarité entre deux chaînes.

Par exemple, si vous souhaitez obtenir un score indiquant à quel point deux noms se ressemblent phonétiquement, vous pourriez :

Calculer d’abord les codes phonétiques avec Double Metaphone pour chacun des noms.
Ensuite, comparer ces codes à l’aide d’une métrique de similarité (par exemple, jaro_winkler_similarity) pour obtenir un score (entre 0 et 1).
Voici un exemple illustrant cette approche :

```python
import jellyfish

def phonetic_similarity(name1: str, name2: str) -> float:
    # Récupère le premier code Double Metaphone pour chacun
    code1 = jellyfish.double_metaphone(name1)[0]
    code2 = jellyfish.double_metaphone(name2)[0]
    
    # Utilise jaro_winkler pour comparer les codes phonétiques obtenus
    return jellyfish.jaro_winkler_similarity(code1, code2)

# Exemple
name1 = "Neige Sinault"
name2 = "Neige Sinno"
score = phonetic_similarity(name1, name2)
print(f"Score de similarité phonétique : {score}")
```